In [1]:
pip install imbalanced-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Copy dataset to avoid modifying original
df_cleaned = pd.read_csv('diabetes.csv')

# Replace 0s in specific columns with their median values (except 'Pregnancies' which can be 0)
columns_with_zeros = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]
imputer = SimpleImputer(strategy="median")
df_cleaned[columns_with_zeros] = imputer.fit_transform(df_cleaned[columns_with_zeros])

# Separate features and target
X = df_cleaned.drop(columns=["Outcome"])
y = df_cleaned["Outcome"]

# Apply feature selection (SelectKBest)
X_selected = SelectKBest(f_classif, k=8).fit_transform(X, y)

# Handle class imbalance with SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_selected, y)

# Split into train-test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Standardize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define optimized MLP model
model = Sequential([
    Dense(64, activation="relu", kernel_regularizer=l2(0.001), input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation="relu", kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation="relu", kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(1, activation="sigmoid")
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.0005), loss="binary_crossentropy", metrics=["accuracy"])

# Early Stopping to prevent overfitting
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

# Train model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=64, callbacks=[early_stopping])

# Evaluate model
test_loss, test_acc = model.evaluate(X_test, y_test)
test_acc_percentage = test_acc * 100

test_acc_percentage


Epoch 1/100
13/13 [==============================] - 1s 16ms/step - loss: 1.0457 - accuracy: 0.5425 - val_loss: 0.8539 - val_accuracy: 0.6250
Epoch 2/100
13/13 [==============================] - 0s 4ms/step - loss: 0.8922 - accuracy: 0.6237 - val_loss: 0.8181 - val_accuracy: 0.7550
Epoch 3/100
13/13 [==============================] - 0s 4ms/step - loss: 0.7662 - accuracy: 0.7075 - val_loss: 0.7969 - val_accuracy: 0.7350
Epoch 4/100
13/13 [==============================] - 0s 4ms/step - loss: 0.7560 - accuracy: 0.7200 - val_loss: 0.7817 - val_accuracy: 0.7300
Epoch 5/100
13/13 [==============================] - 0s 4ms/step - loss: 0.7360 - accuracy: 0.7362 - val_loss: 0.7703 - val_accuracy: 0.7250
Epoch 6/100
13/13 [==============================] - 0s 4ms/step - loss: 0.7219 - accuracy: 0.7425 - val_loss: 0.7613 - val_accuracy: 0.7250
Epoch 7/100
13/13 [==============================] - 0s 4ms/step - loss: 0.7242 - accuracy: 0.7312 - val_loss: 0.7492 - val_accuracy: 0.7250
Epoch 8/100


78.50000262260437